In [1]:
import torch
import numpy as np
import networkx as nx

In [2]:
number_of_nodes = 5
dimensions = 3
clusters = 3
eta = 5.
lower_control = 10 ** -15
alpha = 0.05
lambd = 0.2
beta = 0.05

A = np.random.randint(0, 2, size=(number_of_nodes, number_of_nodes))
A = (((A + A.T) / 2) > 0) * 1.
np.fill_diagonal(A, 1.)
graph = nx.from_numpy_matrix(A)
# np.fill_diagonal(A, 1.)
A = torch.from_numpy(A)

In [3]:
def _modularity_matrix(graph):
    """Calculate modularity matrix."""
    
    degs = nx.degree(graph)

    num_edges = graph.number_of_edges()
    n_count = graph.number_of_nodes()
    mod_matrix = np.zeros((n_count, n_count))
    
    indices_1 = np.array([edge[0] for edge in graph.edges()] + [edge[1] for edge in graph.edges()])
    indices_2 = np.array([edge[1] for edge in graph.edges()] + [edge[0] for edge in graph.edges()])

    scores = [float(degs[e[0]] * degs[e[1]]) / (2 * num_edges) for e in graph.edges()]
    scores += [float(degs[e[1]] * degs[e[0]]) / (2 * num_edges) for e in graph.edges()]

    mod_matrix[indices_1, indices_2] = scores

    return mod_matrix


def modularity_matrix(A):
    """Calculate modularity matrix."""
    
    degs = A.sum(dim=1) + 1
    num_edges = A.tril().sum(dim=(-2, -1))
    batch_size = A.shape[0]

    edge_indices = torch.stack(A.triu().nonzero().transpose(-2, -1)[1:, ...].tensor_split(batch_size, dim=-1))
    row_idx = edge_indices[:, 0, :].view(batch_size, -1).expand(-1, -1)
    col_idx = edge_indices[:, 1, :].view(batch_size, -1).expand(-1, -1)

    scores_l = (torch.gather(degs, -1, row_idx) * torch.gather(degs, -1, col_idx)) / (2 * num_edges.unsqueeze(-1)) 
    scores_u = (torch.gather(degs, -1, col_idx) * torch.gather(degs, -1, row_idx)) / (2 * num_edges.unsqueeze(-1)) 
    
    mod_matrix = torch.zeros_like(A)
    mod_matrix[:, row_idx, col_idx] = scores_l
    mod_matrix[:, col_idx, row_idx] = scores_u

    return mod_matrix

In [4]:
_M = np.random.uniform(0, 1, (number_of_nodes, dimensions))
_U = np.random.uniform(0, 1, (number_of_nodes, dimensions))
_H = np.random.uniform(0, 1, (number_of_nodes, clusters))
_C = np.random.uniform(0, 1, (clusters, dimensions))
_B1 = nx.adjacency_matrix(graph, nodelist=range(graph.number_of_nodes()))
_B2 = _modularity_matrix(graph)
_X = np.transpose(_U)
_overlaps = _B1.dot(_B1)
_S = _B1 + eta * _B1 * (_overlaps)

M = torch.from_numpy(_M).unsqueeze(0).repeat_interleave(2, 0)
U = torch.from_numpy(_U).unsqueeze(0).repeat_interleave(2, 0)
H = torch.from_numpy(_H).unsqueeze(0).repeat_interleave(2, 0)
C = torch.from_numpy(_C).unsqueeze(0).repeat_interleave(2, 0)
B1 = A.unsqueeze(0).repeat_interleave(2, 0)
B2 = modularity_matrix(A.unsqueeze(0))
X = U.transpose(dim0=-2, dim1=-1)
overlaps = B1.matmul(B1)
S = B1 + eta * B1.matmul(overlaps)

print(torch.allclose(B1[0], torch.tensor(_B1.todense())))
print(torch.allclose(B2[0], torch.tensor(_B2)))
print(torch.allclose(X[0], torch.tensor(_X)))
print(torch.allclose(overlaps[0], torch.tensor(_overlaps.todense())))
print(torch.allclose(S[0], torch.tensor(_S.todense())))

True
True
True
True
True


In [5]:
def _update_M(_S, _U, _M, lower_control):
    _enum = _S.dot(_U)
    _denom = np.dot(_M, np.dot(np.transpose(_U), _U))
    _denom[_denom < lower_control] = lower_control
    _new_M = np.multiply(_M, _enum / _denom)
    _row_sums = _new_M.sum(axis=1)
    _new_M = _new_M / _row_sums[:, np.newaxis]
    return _new_M

def update_M(S, U, M, lower_control):
    enum = S.matmul(U)
    denom = M.matmul(U.transpose(-2, -1).matmul(U))
    denom[denom < lower_control] = lower_control
    new_M = M * (enum / denom)
    row_sums = new_M.sum(dim=-1)
    new_M = new_M / row_sums.unsqueeze(-1)
    return new_M

torch.allclose(update_M(S, U, M, lower_control)[0], torch.tensor(_update_M(_S, _U, _M, lower_control)))

True

In [6]:
def _update_U(_S, _U, _M, _C, alpha, lower_control):
    """Update matrix U."""
    enum = _S.dot(_M) + alpha * np.dot(_H, _C)
    denom = np.dot(_U, np.dot(np.transpose(_M), _M) + alpha * np.dot(np.transpose(_C), _C))
    denom[denom < lower_control] = lower_control
    _new_U = np.multiply(_U, enum / denom)
    row_sums = _U.sum(axis=1)
    _new_U = _new_U / row_sums[:, np.newaxis]
    return _new_U


def update_U(S, U, M, C, alpha, lower_control):
    """Update matrix U."""
    enum = S.matmul(M) + alpha * H.matmul(C)
    denom = U.matmul(M.transpose(-2, -1).matmul(M) + (alpha * C.transpose(-2, -1).matmul(C)))
    denom[denom < lower_control] = lower_control
    new_U = U * (enum / denom)
    row_sums = U.sum(dim=-1)
    new_U = new_U / row_sums.unsqueeze(-1)
    return new_U

torch.allclose(update_U(S, U, M, C, alpha, lower_control)[0], torch.tensor(_update_U(_S, _U, _M, _C, alpha, lower_control)))

True

In [7]:
def _update_C(_U, _H, _C, lower_control):
    """Update matrix C."""
    enum = np.dot(np.transpose(_H), _U)
    denom = np.dot(_C, np.dot(np.transpose(_U), _U))
    denom[denom < lower_control] = lower_control
    frac = enum / denom
    _C = np.multiply(_C, frac)
    row_sums = _C.sum(axis=1)
    _C = _C / row_sums[:, np.newaxis]
    return _C

def update_C(U, H, C, lower_control):
    """Update matrix C."""
    enum = H.transpose(-2, -1).matmul(U)
    denom = C.matmul(U.transpose(-2, -1).matmul(U))
    denom[denom < lower_control] = lower_control
    frac = enum / denom
    C = C * frac
    row_sums = C.sum(dim=-1)
    C = C / row_sums.unsqueeze(-1)
    return C

torch.allclose(update_C(U, H, C, lower_control)[0], torch.tensor(_update_C(_U, _H, _C, lower_control)))

True

In [8]:
def _update_H(_B1, _B2, _H, lambd, beta):
    """Update matrix H."""
    B1H = _B1.dot(_H)
    B2H = _B2.dot(_H)
    HHH = np.dot(_H, (np.dot(np.transpose(_H), _H)))
    UC = np.dot(_U, np.transpose(_C))
    rooted = np.square(2 * beta * B2H) + np.multiply(16 * lambd * HHH, (2 * beta * B1H + 2 * alpha * UC + (4 * lambd - 2 * alpha) * _H))
    rooted[rooted < 0] = 0
    sqroot_1 = np.sqrt(rooted)
    enum = -2 * beta * B2H + sqroot_1
    denom = 8 * lambd * HHH
    denom[denom < lower_control] = lower_control
    rooted = enum / denom
    rooted[rooted < 0] = 0
    sqroot_2 = np.sqrt(rooted)
    _H = np.multiply(_H, sqroot_2)
    row_sums = _H.sum(axis=1)
    _H = _H / row_sums[:, np.newaxis]
    return _H

def update_H(B1, B2, H, lambd, beta):
    """Update matrix H."""
    B1H = B1.matmul(H)
    B2H = B2.matmul(H)
    HHH = H.matmul(H.transpose(-2, -1).matmul(H))
    UC = U.matmul(C.transpose(-2, -1))
    rooted = (2 * beta * B2H).pow(2) + (16 * lambd * HHH * (2 * beta * B1H + 2 * alpha * UC + (4 * lambd - 2 * alpha) * H))
    rooted[rooted < 0] = 0
    sqroot_1 = rooted.sqrt()
    enum = -2 * beta * B2H + sqroot_1
    denom = 8 * lambd * HHH
    denom[denom < lower_control] = lower_control
    rooted = enum / denom
    rooted[rooted < 0] = 0
    sqroot_2 = rooted.sqrt()
    H = H * sqroot_2
    row_sums = H.sum(dim=-1)
    H = H / row_sums.unsqueeze(-1)
    return H

torch.allclose(update_H(B1, B2, H, lambd, beta)[0], torch.tensor(_update_H(_B1, _B2, _H, lambd, beta)))

True

In [33]:
class MNMF:
    """Non-negative matrix factorization with modularity maximisation 
    https://dl.acm.org/doi/10.5555/3298239.3298270
    https://github.com/benedekrozemberczki/karateclub/blob/master/karateclub/community_detection/overlapping/mnmf.py
    """

    def __init__(self, dimensions=128, clusters=4, lambd=0.2, alpha=0.05, beta=0.05,
                 eta=5.0, iterations=200, epsilon=1e-10, seed=1234):

        self.number_of_nodes = number_of_nodes
        self.dimensions = dimensions
        self.clusters = clusters
        self.lambd = lambd
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.epsilon = epsilon
        self.eta = eta
        self.seed = seed


    def _initialize_parameters(self):

        batch_size = self.A.shape[0]
        number_of_nodes = self.A.shape[-1]
        kwargs = dict(dtype=self.A.dtype, device=self.A.device)

        # https://stackoverflow.com/questions/50411191/how-to-compute-the-cosine-similarity-in-pytorch-for-all-rows-in-a-matrix-with-re
        A_norm = self.A / self.A.norm(dim=-1).unsqueeze(-1)
        cos_sim = A_norm.matmul(A_norm.transpose(-2, -1))
        
        self.M = torch.rand(batch_size, number_of_nodes, self.dimensions, **kwargs)
        self.U = torch.rand(batch_size, number_of_nodes, self.dimensions, **kwargs)
        self.H = torch.rand(batch_size, number_of_nodes, self.clusters, **kwargs)
        self.C = torch.rand(batch_size, self.clusters, self.dimensions, **kwargs)
        self.B1 = self._compute_B1()
        self.S = self.A + self.eta * cos_sim


    def _compute_B1(self):
        """Calculate modularity matrix."""
        
        degs = self.A.sum(dim=1) + 1
        num_edges = self.A.tril().sum(dim=(-2, -1))
        batch_size = self.A.shape[0]

        edge_indices = torch.stack(self.A.triu().nonzero().transpose(-2, -1)[1:, ...].tensor_split(batch_size, dim=-1))
        row_idx = edge_indices[:, 0, :].view(batch_size, -1).expand(-1, -1)
        col_idx = edge_indices[:, 1, :].view(batch_size, -1).expand(-1, -1)

        scores_l = (torch.gather(degs, -1, row_idx) * torch.gather(degs, -1, col_idx)) / (2 * num_edges.unsqueeze(-1)) 
        scores_u = (torch.gather(degs, -1, col_idx) * torch.gather(degs, -1, row_idx)) / (2 * num_edges.unsqueeze(-1)) 
        
        B1 = torch.zeros_like(self.A)
        B1[:, row_idx, col_idx] = scores_l
        B1[:, col_idx, row_idx] = scores_u

        return B1


    def _update_M(self): 
        """Update matrix M."""

        enum = self.S.matmul(self.U)
        denom = self.M.matmul(self.U.transpose(dim0=-2, dim1=-1).matmul(self.U))
        denom[denom < self.epsilon] = self.epsilon
        self.M = self.M * (enum / denom)
        row_sums = self.M.sum(dim=-1)
        self.M = self.M / row_sums.unsqueeze(-1)
     

    def _update_U(self):
        """Update matrix U."""

        enum = self.S.matmul(self.M) + self.alpha * self.H.matmul(self.C)
        denom = self.U.matmul(self.M.transpose(dim0=-2, dim1=-1).matmul(self.M) + (self.alpha * self.C.transpose(dim0=-2, dim1=-1).matmul(self.C)))
        denom[denom < self.epsilon] = self.epsilon
        self.U = self.U * (enum / denom)
        row_sums = self.U.sum(dim=-1)
        self.U = self.U / row_sums.unsqueeze(-1)
  

    def _update_C(self):
        """Update matrix C."""

        enum = self.H.transpose(dim0=-2, dim1=-1).matmul(self.U)
        denom = self.C.matmul(self.U.transpose(dim0=-2, dim1=-1).matmul(self.U))
        denom[denom < self.epsilon] = self.epsilon
        self.C = self.C * (enum / denom)
        row_sums = self.C.sum(dim=-1)
        self.C = self.C / row_sums.unsqueeze(-1)


    def _update_H(self):
        """Update matrix H."""

        AH = self.A.matmul(self.H)
        B1H = self.B1.matmul(self.H)
        HHH = self.H.matmul(self.H.transpose(dim0=-2, dim1=-1).matmul(self.H))
        UC = self.U.matmul(self.C.transpose(dim0=-2, dim1=-1))
        
        rooted = (2 * self.beta * B1H).pow(2)  
        rooted += (16 * self.lambd * HHH * (2 * self.beta * AH + 2 * self.alpha * UC + (4 * self.lambd - 2 * self.alpha) * self.H))
        rooted[rooted < 0] = 0
        sqroot_1 = rooted.sqrt()
        
        enum = -2 * self.beta * B1H + sqroot_1
        denom = 8 * self.lambd * HHH
        denom[denom < self.epsilon] = self.epsilon
        
        rooted = enum / denom
        rooted[rooted < 0] = 0
        sqroot_2 = rooted.sqrt()
        
        self.H = self.H * sqroot_2
        row_sums = self.H.sum(dim=-1)
        self.H = self.H / row_sums.unsqueeze(-1)

    
    def _compute_loss(self):

        loss = torch.linalg.matrix_norm(self.S - self.M.matmul(self.U.transpose(dim0=-2, dim1=-1))).pow(2)
        loss += self.alpha * torch.linalg.matrix_norm(self.H - self.U.matmul(self.C.transpose(dim0=-2, dim1=-1))).pow(2)
        loss -= self.beta * ((self.H.transpose(dim0=-2, dim1=-1).matmul((self.A - self.B1))).matmul(self.H)).diagonal(offset=0, dim1=-1, dim2=-2).sum(dim=-1)

        return loss
        

    def node_community_membership(self):
        """Get community membership of nodes."""

        # indices = self.H, axis=1)
        # memberships = {i: membership for i, membership in enumerate(indices)}
        # return memberships


    def node_embedding(self):
        """Get node embedding."""
        
        embedding = self.U
        return embedding


    def community_embedding(self):
        """Get community embedding."""

        embedding = self.C
        return embedding


    def fit(self, A):
        """Fit M-NMF clustering model."""

        # self._set_seed()

        self.A = A

        self._initialize_parameters()

        for _iter in range(self.iterations):

            loss = self._compute_loss()

            self._update_M()
            self._update_U()
            self._update_C()
            self._update_H()

            print(loss - self._compute_loss())

In [34]:
data_path = "../data/data.npz"
data_dic = dict(np.load(data_path, allow_pickle=True).items())
A = torch.from_numpy(data_dic["A"]).float()
C = torch.from_numpy(data_dic["C"]).float()

In [35]:
_A = A[0, 1:2, ...]
_C = C[0, 1:2, ...]

print(_A.shape)
print(_C.shape)

torch.Size([1, 100, 100])
torch.Size([1, 100, 4])


In [36]:
model = MNMF()
model.fit(_A)

tensor([9678862.])
tensor([19.9844])
tensor([85.6699])
tensor([126.2949])
tensor([72.2246])
tensor([38.6855])
tensor([25.2539])
tensor([18.8828])
tensor([14.9746])
tensor([12.1270])
tensor([9.8691])
tensor([8.1777])
tensor([6.9434])
tensor([6.0156])
tensor([5.2266])
tensor([4.5645])
tensor([4.0371])
tensor([3.6562])
tensor([3.3164])
tensor([3.0273])
tensor([2.7773])
tensor([2.5996])
tensor([2.4688])
tensor([2.3340])
tensor([2.2109])
tensor([2.1055])
tensor([2.0098])
tensor([1.9297])
tensor([1.8496])
tensor([1.7656])
tensor([1.6738])
tensor([1.5996])
tensor([1.5332])
tensor([1.4785])
tensor([1.4512])
tensor([1.3984])
tensor([1.3652])
tensor([1.3223])
tensor([1.2832])
tensor([1.2500])
tensor([1.2148])
tensor([1.1777])
tensor([1.1445])
tensor([1.0977])
tensor([1.0723])
tensor([1.0332])
tensor([0.9941])
tensor([0.9648])
tensor([0.9316])
tensor([0.8887])
tensor([0.8496])
tensor([0.7930])
tensor([0.7402])
tensor([0.6875])
tensor([0.6270])
tensor([0.5664])
tensor([0.5117])
tensor([0.4551])
te

In [30]:
model.H.argmax(-1)

tensor([[3, 3, 1, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3,
         3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]])

In [32]:
print(_C.argmax(dim=-1))

tensor([[0, 0, 3, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0,
         0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3]])
